# Whitebox

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-312/blob/main/book/geospatial/whitebox.ipynb)

## Overview

In this lecture, we will explore the use of [**WhiteboxTools**](https://github.com/jblindsay/whitebox-tools), a powerful open-source library for performing geospatial analysis. Specifically, we will focus on two key applications: **watershed analysis** and **LiDAR data analysis**. You will learn how to manipulate geospatial data using Python, conduct hydrological analysis, and derive digital elevation models (DEMs) and canopy height models (CHMs) from LiDAR data.

This lecture is structured into two main sections:
1. **Watershed Analysis**: Using DEMs and hydrological tools to delineate watersheds, calculate flow accumulation, and extract stream networks.
2. **LiDAR Data Analysis**: Processing LiDAR point cloud data to derive DEMs, DSMs, and CHMs while removing outliers and improving data quality.

By the end of this session, you will have gained hands-on experience with **WhiteboxTools** and **leafmap**, allowing you to perform a wide range of geospatial and hydrological analyses.

## Learning Objectives

By the end of this lecture, you will be able to:
- Install and configure **WhiteboxTools** and **leafmap** for geospatial analysis.
- Create interactive maps to visualize basemaps and geospatial datasets.
- Perform watershed analysis by delineating watersheds, flow directions, and stream networks.
- Manipulate and analyze Digital Elevation Models (DEMs) to conduct hydrological modeling.
- Process and analyze LiDAR data to generate **Digital Surface Models (DSMs)**, **Digital Elevation Models (DEMs)**, and **Canopy Height Models (CHMs)**.
- Integrate **WhiteboxTools** with Python workflows to automate geospatial analysis.


## Introduction

Below is a brief introduction to Whitebox, a powerful open-source library for geospatial analysis. For more information, please refer to the whiteboxgeo website: https://www.whiteboxgeo.com.

### What is Whitebox?

Whitebox is geospatial data analysis software originally developed at the University of Guelph‘s Geomorphometry and Hydrogeomatics Research Group ([GHRG](https://jblindsay.github.io/ghrg/index.html)) directed by Dr. John Lindsay. Whitebox contains over 550 tools for processing many types of geospatial data. It has many great features such as its extensive use of parallel computing, it doesn’t need other libraries to be installed (e.g., GDAL), it can be used from scripting environments, and it easily plugs into other geographical information system (GIS) software. The Whitebox Toolset Extension provides even more power.

### What can Whitebox do?

Whitebox can be used to perform common GIS and remote sensing analysis tasks. Whitebox also contains advanced tooling for spatial hydrological analysis and LiDAR data processing. Whitebox is not a cartographic or spatial data visualization package; instead it’s meant to serve as an analytical back-end for other data visualization software, like QGIS and ArcGIS. 

### How is Whitebox different?

Whitebox doesn’t compete with QGIS, ArcGIS/Pro, and ArcPy but rather it extends them. You can plug WhiteboxTools into QGIS and ArcGIS and it’ll provide hundreds of additional tools for analyzing all kinds of geospatial data. You can also call Whitebox functions from Python scripts using [Whitebox Workflows](https://www.whiteboxgeo.com/whitebox-workflows-for-python) (WbW). Combine WbW with ArcPy to more effectively automate your data analysis workflows and streamline your geoprocessing solutions.

There are many tools in Whitebox that you won’t find elsewhere. You can think of Whitebox as a portable, cross-platform GIS analysis powerhouse, allowing you to extend your preferred GIS or to embed Whitebox capabilities into your automated scripted workflows. Oh, and it’s fast, really fast!


## Useful Resources for Whitebox

- GitHub Repository: https://github.com/jblindsay/whitebox-tools
- WhiteboxGeo: https://www.whiteboxgeo.com
- User Manual: https://whiteboxgeo.com/manual/wbt_book/preface.html
- Whitebox Workflows for Python: https://www.whiteboxgeo.com/whitebox-workflows-for-python
- Whitebox Workflows for Python Pro: https://www.whiteboxgeo.com/whitebox-workflows-professional
- Python Frontend: https://github.com/opengeos/whitebox-python
- Jupyter Frontend: https://github.com/opengeos/whiteboxgui
- R Frontend: https://github.com/opengeos/whiteboxR
- ArcGIS Toolbox: https://github.com/opengeos/WhiteboxTools-ArcGIS
- QGIS Plugin: https://plugins.qgis.org/plugins/whitebox_workflows_for_qgis

## Installation

To get started, we need to install the required packages, such as `leafmap`, and `whitebox`. Uncomment the following lines to install the packages.

In [ ]:
# %pip install "leafmap[raster]" "leafmap[lidar]" mapclassify

In [ ]:
# %pip install numpy==1.26.4

## Import libraries

In [1]:
import os
import leafmap
import numpy as np

Some of the raster datasets generated by whitebox will be int32 type with a nodata value of -32768. To make it easier to visualize these datasets, we set the nodata value as an environment variable, which will be used by leafmap to set the nodata value when visualizing the raster datasets.

In [2]:
os.environ["NODATA"] = "-32768"

## Part 1: Watershed Analysis

### Create Interactive Maps

To perform watershed analysis, we first create an interactive map using **leafmap**. This step allows us to visualize different basemaps.

In [3]:
m = leafmap.Map()
m.add_basemap("OpenTopoMap")
m.add_basemap("USGS 3DEP Elevation")
m.add_basemap("USGS Hydrography")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

### Download Watershed Data

Next, we download watershed data for the **Calapooia River basin** in Oregon. We'll use the latitude and longitude of a point in the basin to extract watershed boundary data.

In [4]:
lat = 44.361169
lon = -122.821802

m = leafmap.Map(center=[lat, lon], zoom=10) # White Mountains, NH
m.add_marker([lat, lon])
m

Map(center=[44.361169, -122.821802], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

Download the watershed data and visualize it:

In [5]:
geometry = {"x": lon, "y": lat}

In [6]:
gdf = leafmap.get_wbd(geometry, digit=10, return_geometry=True)  # downloads USGS Watershed Boundary Dataset (HUC‑10)
gdf.explore()

Save the watershed boundary to a **GeoJSON** file:

In [7]:
gdf.to_file("data/basin.geojson")

### Download and Display DEM

We download a **Digital Elevation Model (DEM)** from the USGS 3DEP Elevation service to analyze the terrain of the watershed. The DEM will be used to delineate watersheds, calculate flow accumulation, and extract stream networks. The `leafmap.get_3dep_dem()` function returns the DEM as an `xarray.DataArray` object. Optionally, you can save the DEM to a GeoTIFF file by setting the `output` parameter.

In [8]:
array = leafmap.get_3dep_dem(
    gdf,
    resolution=30,
    output="dem.tif",
    dst_crs="EPSG:3857",
    to_cog=True,
    overwrite=True,
)
array

Reading input: /Users/xiaozhongsun/Desktop/geog346_s25/dem.tif

Adding overviews...
Updating dataset tags...
Writing output to: /Users/xiaozhongsun/Desktop/geog346_s25/dem.tif


<xarray.DataArray 'elevation' (y: 2488, x: 2727)> Size: 27MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]],
      shape=(2488, 2727), dtype=float32)
Coordinates:
  * x            (x) float64 22kB -1.371e+07 -1.371e+07 ... -1.361e+07
  * y            (y) float64 20kB 5.565e+06 5.565e+06 ... 5.478e+06 5.478e+06
    band         int64 8B 1
    spatial_ref  int64 8B 0
Attributes:
    scale_factor:         1.0
    add_offset:           0.0
    units:                m
    vertical_datum:       NAVD88
    vertical_resolution:  0.001
    _FillValue:           nan

Visualize the DEM on the map:

In [ ]:
# $pip install localtileserver ipyleaflet


In [ ]:
m.add_raster("dem.tif", palette="terrain", nodata=np.nan, layer_name="DEM")
m

Map(bottom=95254.0, center=[44.360913999999994, -122.73361600000001], controls=(ZoomControl(options=['position…

Exception possibly due to cache backend.
Traceback (most recent call last):
  File "/Users/xiaozhongsun/miniconda3/envs/geog346/lib/python3.12/site-packages/flask_caching/__init__.py", line 435, in decorated_function
    self.cache.set(
  File "/Users/xiaozhongsun/miniconda3/envs/geog346/lib/python3.12/site-packages/cachelib/simple.py", line 75, in set
    self._prune()
  File "/Users/xiaozhongsun/miniconda3/envs/geog346/lib/python3.12/site-packages/cachelib/simple.py", line 52, in _prune
    self._remove_expired(now)
  File "/Users/xiaozhongsun/miniconda3/envs/geog346/lib/python3.12/site-packages/cachelib/simple.py", line 36, in _remove_expired
    toremove = [k for k, (expires, _) in self._cache.items() if expires < now]
                                         ^^^^^^^^^^^^^^^^^^^
RuntimeError: dictionary keys changed during iteration


### Get DEM metadata

We can get the metadata of the DEM, such as the spatial resolution, bounding box, and coordinate reference system (CRS).

In [10]:
metadata = leafmap.image_metadata("dem.tif")
metadata

{'bounds': {'left': -123.163442,
  'bottom': 44.080546,
  'right': -122.30379,
  'top': 44.641282},
 'crs': 'PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]',
 'band_metadata': [('b1', {})],
 'band_descriptions': [('b1', 'elevation')],
 'dtype': 'float32',
 'nodata_type': 'Nodata',
 'colorinterp': ['gray'],
 'scales': [1.0],
 'offsets': [0.0],
 'colormap': None,
 

Get a summary statistics of the DEM.

### Add colorbar

Add a colorbar to the map to show the elevation values. Use the `image_min_max()` function to get the minimum and maximum values of the DEM.

In [11]:
leafmap.image_min_max("dem.tif")

(69.4413833618164, 1564.8543701171875)

In [12]:
m.add_colormap(cmap="terrain", vmin="60", vmax=1500, label="Elevation (m)")

### Initialize WhiteboxTools

Initialize the WhiteboxTools class.

In [13]:
wbt = leafmap.WhiteboxTools()

Check the WhiteboxTools version.

In [14]:
wbt.version()

"WhiteboxTools v2.4.0 (c) Dr. John Lindsay 2017-2023\n\nWhiteboxTools is an advanced geospatial data analysis platform developed at\nthe University of Guelph's Geomorphometry and Hydrogeomatics Research \nGroup (GHRG). See www.whiteboxgeo.com for more details.\n"

Display the WhiteboxTools interface, which lists all available tools. You can use this interface to search for specific tools. You can also run any tool using the interface. However, we will use the Python API to run the tools.

In [15]:
leafmap.whiteboxgui()

### Set working directory

Set the working directory to save the intermediate and output files. Set `wbt.version=False` to suppress the Whitebox processing log.

In [16]:
# Build the absolute path to your data subfolder
data_dir = os.path.join(os.getcwd(), "data")
wbt.set_working_dir(data_dir)
wbt.verbose = False

In [ ]:
# print("Working dir:", os.getcwd())
# print("Files here:", os.listdir())
# if os.path.isdir("data"):
#     print("data/ contains:", os.listdir("data"))
# else:
#     print("No data/ folder in this cwd")

### Smooth DEM

All WhiteboxTools functions will return 0 if they are successful, and 1 if they are not.

Smoothing the DEM enhances the quality of subsequent hydrological analysis.

In [17]:
wbt.feature_preserving_smoothing("dem.tif", "smoothed.tif", filter=9)

0

Visualize the smoothed DEM and watershed boundary on the map.

Visualize the smoothed DEM:

In [18]:
m = leafmap.Map()
m.add_basemap("Satellite")
m.add_raster("data/smoothed.tif", colormap="terrain", layer_name="Smoothed DEM")
m.add_geojson("data/basin.geojson", layer_name="Watershed", info_mode=None)
m.add_basemap("USGS Hydrography", show=False)
m

Map(center=[44.360913999999994, -122.73361600000001], controls=(ZoomControl(options=['position', 'zoom_in_text…

### Create hillshade

Create a hillshade from the smoothed DEM.

In [19]:
wbt.hillshade("smoothed.tif", "hillshade.tif", azimuth=315, altitude=35)

0

Overlay the hillshade on the smoothed DEM with transparency.

In [20]:
m.add_raster("data/hillshade.tif", layer_name="Hillshade")
m.layers[-1].opacity = 0.6

### Find no-flow cells

Find cells with undefined flow, i.e. no valid flow direction, based on the D8 flow direction algorithm.

In [21]:
wbt.find_no_flow_cells("smoothed.tif", "noflow.tif")

0

Visualize the no-flow cells on the map.

In [22]:
m.add_raster("data/noflow.tif", layer_name="No Flow Cells")

### Fill depressions

First, we fill any depressions in the DEM to ensure proper flow simulation.

In [23]:
wbt.fill_depressions("smoothed.tif", "filled.tif")

0

Alternatively, you can use depression breaching to fill the depressions.

In [24]:
wbt.breach_depressions("smoothed.tif", "breached.tif")

0

In [25]:
wbt.find_no_flow_cells("breached.tif", "noflow2.tif")

0

In [26]:
m.layers[-1].visible = False
m.add_raster("data/noflow2.tif", layer_name="No Flow Cells after Breaching")
m

Map(bottom=24039.0, center=[44.360913999999994, -122.73361600000001], controls=(ZoomControl(options=['position…

### Delineate flow direction

Next, we delineate the flow direction based on the D8 algorithm.

In [27]:
wbt.d8_pointer("breached.tif", "flow_direction.tif")

0

### Calculate flow accumulation

Now, calculate **flow accumulation** to understand how water collects across the landscape.

In [28]:
wbt.d8_flow_accumulation("breached.tif", "flow_accum.tif")

0

In [29]:
m.add_raster("data/flow_accum.tif", layer_name="Flow Accumulation")
m

Map(bottom=95246.0, center=[44.360913999999994, -122.73361600000001], controls=(ZoomControl(options=['position…

### Extract streams

Extract the stream network using the flow accumulation data.

In [30]:
wbt.extract_streams("flow_accum.tif", "streams.tif", threshold=5000)

0

In [31]:
m.layers[-1].visible = False
m.add_raster("data/streams.tif", layer_name="Streams")
m

Map(bottom=190465.0, center=[44.360913999999994, -122.73361600000001], controls=(ZoomControl(options=['positio…

### Calculate distance to outlet

In [32]:
wbt.distance_to_outlet(
    "flow_direction.tif", streams="streams.tif", output="distance_to_outlet.tif"
)

0

In [33]:
m.add_raster("data/distance_to_outlet.tif", layer_name="Distance to Outlet")
m

Map(bottom=95254.0, center=[44.360913999999994, -122.73361600000001], controls=(ZoomControl(options=['position…

### Vectorize streams

In [34]:
wbt.raster_streams_to_vector(
    "streams.tif", d8_pntr="flow_direction.tif", output="streams.shp"
)

0

The raster_streams_to_vector tool has a bug. The output vector file is missing the coordinate system. Use `leafmap.vector_set_crs()` to set the coordinate system.

In [35]:
leafmap.vector_set_crs(source="data/streams.shp", output="data/streams.shp", crs="EPSG:3857")

In [36]:
m.add_shp(
    "data/streams.shp",
    layer_name="Streams Vector",
    style={"color": "#ff0000", "weight": 3, "opacity": 0.2},
    info_mode=None,
)
m

Map(bottom=95339.0, center=[44.27765451038985, -122.43232727050781], controls=(ZoomControl(options=['position'…

You can turn on the USGS Hydrography basemap to visualize the stream network and compare it with the extracted stream network.

### Delineate the longest flow path

You can delineate the longest flow path in the watershed.

In [37]:
wbt.basins("flow_direction.tif", "basins.tif")

0

In [38]:
wbt.longest_flowpath(
    dem="breached.tif", basins="basins.tif", output="longest_flowpath.shp"
)

0

Select only the longest flow path.

In [39]:
leafmap.select_largest(
    "data/longest_flowpath.shp", column="LENGTH", output="data/longest_flowpath.shp"
)

In [40]:
m.add_shp(
    "data/longest_flowpath.shp",
    layer_name="Longest Flowpath",
    style={"color": "#ff0000", "weight": 3},
)
m

Map(bottom=380042.0, center=[44.37933052388131, -122.9392433166504], controls=(ZoomControl(options=['position'…

### Generate a pour point

To delineate a watershed, you need to specify a pour point. You can use the outlet of the longest flow path as the pour point or specify a different point. Use the drawing tool to place a marker on the map to specify the pour point. If no marker is placed, the default pour point below will be used.

In [41]:
if m.user_roi is not None:
    m.save_draw_features("data/pour_point.shp", crs="EPSG:3857")
else:
    lat = 44.284642
    lon = -122.611217
    leafmap.coords_to_vector([lon, lat], output="data/pour_point.shp", crs="EPSG:3857")
    m.add_marker([lat, lon])

### Snap pour point to stream

Snap the pour point to the nearest stream.

In [42]:
wbt.snap_pour_points(
    "pour_point.shp", "flow_accum.tif", "pour_point_snapped.shp", snap_dist=300
)

0

Visualize the snapped pour point on the map.

In [44]:
m.add_shp("data/pour_point_snapped.shp", layer_name="Pour Point", info_mode=False)
m

Map(bottom=95236.0, center=[44.37933052388131, -122.9392433166504], controls=(ZoomControl(options=['position',…

### Delineate watershed

Delineate the watershed using a **pour point** and the flow direction data.

In [45]:
wbt.watershed("flow_direction.tif", "pour_point_snapped.shp", "watershed.tif")

0

Visualize the watershed boundary on the map.

In [46]:
m.add_raster("data/watershed.tif", layer_name="Watershed")
m

Map(bottom=95342.0, center=[44.360913999999994, -122.73361600000001], controls=(ZoomControl(options=['position…

### Convert watershed raster to vector

You can convert the watershed raster to a vector file.

In [47]:
wbt.raster_to_vector_polygons("watershed.tif", "watershed.shp")

0

Visualize the watershed boundary on the map.

In [49]:
m.layers[-1].visible = False
m.add_shp(
    "data/watershed.shp",
    layer_name="Watershed Vector",
    style={"color": "#ffff00", "weight": 3},
    info_mode=False,
)
m

Map(bottom=190499.0, center=[44.218139083121855, -122.59952545166017], controls=(ZoomControl(options=['positio…

## Part 2: LiDAR Data Analysis

In this section, we will process LiDAR data to derive **Digital Surface Models (DSMs)**, **Digital Elevation Models (DEMs)**, and **Canopy Height Models (CHMs)**. We will also remove outliers and improve the quality of the LiDAR data.

### Set up whitebox

First, we set up the WhiteboxTools and leafmap libraries.

In [ ]:
# import leafmap

In [ ]:
# wbt = leafmap.WhiteboxTools()
# wbt.set_working_dir(os.getcwd())
# wbt.verbose = False

### Download a sample dataset

We download a sample LiDAR dataset for **Madison**.

In [52]:
url = "https://github.com/opengeos/datasets/releases/download/lidar/madison.zip"
filename = "data/madison.las"

In [51]:
leafmap.download_file(url, "data/madison.zip", quiet=True)

'/Users/xiaozhongsun/Desktop/geog346_s25/data/madison.zip'

### Read LAS/LAZ data

Load and inspect the LiDAR data:

In [54]:
laz = leafmap.read_lidar(filename)

In [55]:
laz

<LasData(1.3, point fmt: <PointFormat(1, 0 bytes of extra dims)>, 4068294 points, 2 vlrs)>

In [56]:
str(laz.header.version)

'1.3'

### Upgrade file version

Upgrade the LAS file version to 1.4.

In [57]:
las = leafmap.convert_lidar(laz, file_version="1.4")

In [58]:
str(las.header.version)

'1.4'

### Write LAS data

Save the LAS data to a new file.

In [59]:
leafmap.write_lidar(las, "madison.las")

### Histogram analysis

Plot the histogram of the LiDAR data. The histogram shows the distribution of the LiDAR points based on their elevation values. The tool generates a histogram of the LiDAR data and saves it to an HTM file. You can open the HTM file in a web browser to view the histogram.

In [60]:
wbt.lidar_histogram("madison.las", "histogram.html")

0

### Visualize LiDAR data

Run the `view_lidar()` function to visualize the LiDAR data in 3D. Note that the `view_lidar()` function may not work in some environments, such as Google Colab.

The pyvista and pyntcloud packages are required for this function. Use `pip install "leafmap[lidar]"` to install them.

In [62]:
leafmap.view_lidar("madison.las")

Widget(value='<iframe src="http://localhost:62807/index.html?ui=P_0x337251df0_0&reconnect=auto" class="pyvista…

### Remove outliers

Remove outliers from the LiDAR dataset:

In [63]:
wbt.lidar_elevation_slice("madison.las", "madison_rm.las", minz=0, maxz=450)

0

### Visualize LiDAR data after removing outliers

We can visualize the LiDAR data after removing the outliers.

In [65]:
leafmap.view_lidar("data/madison_rm.las", cmap="terrain")

Widget(value='<iframe src="http://localhost:62807/index.html?ui=P_0x33504d4c0_1&reconnect=auto" class="pyvista…

### Create DSM

Using the LiDAR data to create a **Digital Surface Model (DSM)**.

In [66]:
wbt.lidar_digital_surface_model(
    "madison_rm.las", "dsm.tif", resolution=1.0, minz=0, maxz=450
)

0

The DSM generated by whitebox is missing the coordinate system. Use `leafmap.raster_set_crs()` to set the coordinate system.

In [67]:
leafmap.add_crs("data/dsm.tif", epsg=2255)

### Visualize DSM

Visualize the DSM on the map.

In [ ]:
m = leafmap.Map()
m.add_basemap("Satellite")
m.add_raster("data/dsm.tif", colormap="terrain", layer_name="DSM")
m

### Create DEM

We can create a bare-earth DEM from the DSM. The tool is typically applied to LiDAR DEMs which frequently contain numerous off-terrain objects (OTOs) such as buildings, trees and other vegetation, cars, fences and other anthropogenic objects. 

In [1]:
wbt.remove_off_terrain_objects("dsm.tif", "dem.tif", filter=25, slope=15.0)

NameError: name 'wbt' is not defined

### Visualize DEM

Visualize the bear-earth DEM on the map.

In [ ]:
m.add_raster("dem.tif", palette="terrain", layer_name="DEM")
m

### Create CHM


We can a **Canopy Height Model (CHM)** by subtracting the DEM from the DSM.

In [ ]:
chm = wbt.subtract("dsm.tif", "dem.tif", "chm.tif")

Visualize the CHM on the map.

In [ ]:
m.add_raster("chm.tif", palette="gist_earth", layer_name="CHM")
m.add_layer_manager()
m

## Summary

This lecture provided a comprehensive introduction to **WhiteboxTools**, an open-source geospatial analysis library with a focus on hydrological and LiDAR data analysis. Through this session, students learned to install and configure WhiteboxTools in Python, integrate it with **leafmap** for interactive mapping, and apply it to specific geospatial tasks.

Key takeaways from this lecture include:

- **Watershed Analysis**: The lecture covered watershed delineation using Digital Elevation Models (DEMs), including techniques like flow direction, flow accumulation, stream extraction, and watershed boundary delineation.
- **LiDAR Data Processing**: Students were introduced to LiDAR data manipulation, including the derivation of Digital Surface Models (DSMs), Digital Elevation Models (DEMs), and Canopy Height Models (CHMs), along with methods for outlier removal and terrain quality improvement.

By completing the hands-on exercises, students gained practical experience with WhiteboxTools for geospatial processing, preparing them to apply these techniques in varied real-world GIS workflows.
